In [1]:
%matplotlib qt
import mib2hspy as m2h
import pandas as pd


In [2]:
m1 = m2h.Magnification(8000, 12030, '2020-12-07', acceleration_voltage=200000, mode='TEM', camera='Ultrascan', mag_mode='SAMAG')
cl1 = m2h.Cameralength(8, 16.2, '2020-12-07', scale=0.00134, spot_size=0.5, acceleration_voltage=200000, camera='Merlin')
cl2 = m2h.Cameralength(10, 19.3, '2020-12-07', acceleration_voltage=200000, camera='Merlin')
cl3 = m2h.Cameralength(10, 19.3, '2020-12-07', acceleration_voltage=200000, camera='Ultrascan')
step1 = m2h.StepSize(2.5, 2.4, '2020-12-07', direction='X', mode='STEM', alpha=None)
precession_calibration = m2h.PrecessionAngle(1.0, 1.04, 20., '2020-12-07', mode='NBD', alpha='Alpha 4', deflectors={'Upper_1': {'X': {'A':0.1, 'P':39}, 'Y': {'A': 0.5, 'P': 44}}})
spot1 = m2h.Spotsize(3, '2020-12-07', spot_size=0.5, mode='NBD', alpha=4, acceleration_voltage=200000)
calibrations = m2h.CalibrationList()
calibrations+=m1
calibrations+=cl1
calibrations+=cl2
calibrations+=cl3
calibrations+=step1
calibrations += precession_calibration
calibrations += spot1
#print([calibration.nominal_value for calibration in calibrations])
print(calibrations.dataframe)


  Label  Nominal Magnification ()  Magnification ()        Date  \
0   IMG                    8000.0           12030.0  2020-12-07   
1  DIFF                       NaN               NaN  2020-12-07   
2  DIFF                       NaN               NaN  2020-12-07   
3  DIFF                       NaN               NaN  2020-12-07   
4  STEP                       NaN               NaN  2020-12-07   
5  PREC                       NaN               NaN  2020-12-07   
6  SPOT                       NaN               NaN  2020-12-07   

   Acceleration Voltage (V)  Mode Mag mode     Camera  \
0                  200000.0   TEM    SAMAG  Ultrascan   
1                  200000.0   NaN      NaN     Merlin   
2                  200000.0   NaN      NaN     Merlin   
3                  200000.0   NaN      NaN  Ultrascan   
4                       NaN  STEM      NaN        NaN   
5                       NaN   NBD      NaN        NaN   
6                  200000.0   NBD      NaN        NaN   

   Nom

In [5]:
cl1.as_dataframe()
#spot1.as_dataframe()

,Label,Nominal Cameralength (cm),Cameralength (cm),Date,Acceleration Voltage (V),Nominal Spotsize (nm),Camera,Scale (mrad),Scale (deg)
0,DIFF,8.0,16.2,2020-12-07,200000,0.5,Merlin,0.033606,0.001926


In [ ]:
new_calibrations = m2h.generate_from_dataframe(calibrations.dataframe)
print(new_calibrations.dataframe)

In [5]:
precession_calibration.as_dataframe(ignore_nans=False)

,Label,Nominal Precession Angle (deg),Precession Angle (deg),Date,Mode,Alpha,Precession excitation (%),Deflector Upper 1 X Amplitude (%),Deflector Upper 1 X Phase (deg),Deflector Upper 1 Y Amplitude (%),...,Deflector Upper 2 Y Amplitude (%),Deflector Upper 2 Y Phase (deg),Deflector Lower 1 X Amplitude (%),Deflector Lower 1 X Phase (deg),Deflector Lower 1 Y Amplitude (%),Deflector Lower 1 Y Phase (deg),Deflector Lower 2 X Amplitude (%),Deflector Lower 2 X Phase (deg),Deflector Lower 2 Y Amplitude (%),Deflector Lower 2 Y Phase (deg)
0,PREC,1.0,1.04,2020-12-07,NBD,Alpha 4,20.0,0.1,39.0,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
step1 = m2h.StepSize(2, 1.9, '2020-12-07', mode='STEM', alpha=None)
print(step1.as_dataframe())

In [ ]:
cl3.scale

In [ ]:
print(calibrations['`Nominal Cameralength (cm)`==8.0'])

In [2]:

#print(precession_calibration)
print(precession_calibration.as_dataframe(ignore_nans=False))

  Label  Nominal Precession Angle (deg)  Precession Angle (deg)        Date  \
0  PREC                             1.0                    1.04  2020-12-07   

  Mode    Alpha  Precession excitation (%)  Deflector Upper 1 X Amplitude (%)  \
0  NBD  Alpha 4                       20.0                                NaN   

   Deflector Upper 1 X Phase (deg)  Deflector Upper 1 Y Amplitude (%)  ...  \
0                              NaN                                NaN  ...   

   Deflector Upper 2 Y Amplitude (%)  Deflector Upper 2 Y Phase (deg)  \
0                                NaN                              NaN   

   Deflector Lower 1 X Amplitude (%)  Deflector Lower 1 X Phase (deg)  \
0                                NaN                              NaN   

   Deflector Lower 1 Y Amplitude (%)  Deflector Lower 1 Y Phase (deg)  \
0                                NaN                              NaN   

   Deflector Lower 2 X Amplitude (%)  Deflector Lower 2 X Phase (deg)  \
0      

In [ ]:
from mib2hspy import DiffractionScale

In [ ]:
cl = DiffractionScale(0.000135)
print(cl)
print(cl.to_inv_nm(200000))
print(cl.calculate_cameralength(200000, 55E-6))
dp = m2h.Cameralength(8, 16.20, '2020-11-20', scale=cl, acceleration_voltage=200000)

In [ ]:
print(dp.as_dataframe())

In [ ]:
mag1 = m2h.Magnification(8000, 16302, '2020-12-01', scale=m2h.Scale(0.34, 'nm'))
mag2 = m2h.Magnification(8000, 16302, '2020-12-01')
cl1 = m2h.Cameralength(8, 16.32, '2020-12-02', scale=m2h.Scale(0.34, '1/nm'), Acceleration_voltage=200000)

df = pd.DataFrame()
df = mag1.add_to_dataframe(df)
df = mag2.add_to_dataframe(df)
df = cl1.add_to_dataframe(df)

print(df)

In [12]:
cl = DiffractionScale(0.000135)
print(cl)
print(cl.to_inv_nm(200000))
print(cl.calculate_cameralength(200000, 55E-6))
dp = m2h.Cameralength(8, 16.20, '2020-11-20', scale=cl, acceleration_voltage=200000)

DiffractionScale Scale=0.000135 1/Å
DiffractionScale Scale=1.35e-05 1/nm
16.24474187833144


In [13]:
print(dp.as_dataframe())

   Nominal Cameralength (cm)  Cameralength (cm)  Scale (1/Å)  Scale (mrad)  \
0                        8.0               16.2     0.000135      0.003386   

   Scale (deg)        Date  Acceleration_voltage  Mode Mag_mode Alpha  Spot  \
0     0.000194  2020-11-20                200000  None     None  None  None   

  Spot_size Camera Microscope  
0      None   None       None  


In [3]:
mag1 = m2h.Magnification(8000, 16302, '2020-12-01', scale=m2h.Scale(0.34, 'nm'))
mag2 = m2h.Magnification(8000, 16302, '2020-12-01')
cl1 = m2h.Cameralength(8, 16.32, '2020-12-02', scale=m2h.Scale(0.34, '1/nm'), Acceleration_voltage=200000)

df = pd.DataFrame()
df = mag1.add_to_dataframe(df)
df = mag2.add_to_dataframe(df)
df = cl1.add_to_dataframe(df)

print(df)

   Nominal Magnification ()  Magnification ()  Scale (nm)        Date  \
0                    8000.0           16302.0        0.34  2020-12-01   
1                    8000.0           16302.0         NaN  2020-12-01   
2                       NaN               NaN         NaN  2020-12-02   

  Acceleration_voltage  Mode Mag_mode Alpha  Spot Spot_size Camera Microscope  \
0                 None  None     None  None  None      None   None       None   
1                 None  None     None  None  None      None   None       None   
2               200000  None     None  None  None      None   None       None   

   Scale (nm/px)  Nominal Cameralength (cm)  Cameralength (cm)  Scale (1/nm)  \
0            NaN                        NaN                NaN           NaN   
1            NaN                        NaN                NaN           NaN   
2            NaN                        8.0              16.32          0.34   

   Scale (mrad)  Scale (deg)  
0           NaN          NaN  

In [3]:
df = mag1.as_dataframe()
df = mag2.add_to_dataframe(df)
print(df)


   Nominal Magnification ()  Magnification ()        Date  \
0                    8000.0           16302.0  2020-12-01   
1                    8000.0           16302.0  2020-12-02   

  Acceleration_voltage  Mode Mag_mode Alpha  Spot Spot_size Camera  
0                 None  None     None  None  None      None   None  
1                 None  None     None  None  None      None   None  


In [4]:
df = mag1.as_dataframe()
df = mag2.add_to_dataframe(df, remove_duplicates=True)
print(df)
#print(mag)

   Nominal {self.name} ({self.units})  {self.name} ({self.units})        Date  \
0                              8000.0                     16302.0  2020-12-02   
1                              8000.0                     16302.0  2020-12-01   

  Acceleration_voltage  Mode Mag_mode Alpha  Spot Spot_size Camera  
0                 None  None     None  None  None      None   None  
1                 None  None     None  None  None      None   None  


In [ ]:
microscope = m2h.Microscope()

print(microscope)

In [2]:
microscope.set_acceleration_voltage(200)
microscope.set_alpha(4)
microscope.set_mode('NBD')
microscope.set_nominal_cameralength(8)
microscope.set_cameralength(16.2)

print(microscope)
print(microscope.get_defined_parameters_(as_dict=True))

Parameter           Value     Units    Nominal value
------------------  --------  -------  ---------------
HT                  200000.0  V
Mode                NBD
Alpha               4
Magnification       nan                nan
Camera length       16.2      cm       8
Magnification Mode
Rocking angle       nan       deg      nan
Rocking frequency   nan       Hz
Step Y              nan       nm       nan
Step X              nan       nm       nan
Convergence angle   nan       mrad     nan
Condenser aperture  nan       um       nan
Spot                nan
Spotsize            nan       nm       nan
Acquisition Date
{'ht': {'Value': 200000.0}, 'mode': {'Value': 'NBD'}, 'alpha': {'Value': 4}, 'camera_length': {'nominal_value': 8, 'actual_value': 16.2}}
